In [ ]:
import requests
from tqdm.auto import tqdm
import pandas as pd
import re, os, csv


TOKEN = os.environ.get('TOKEN') 
VERSION = '5.130'


# Выкачивание мета-информации

In [ ]:
filename = '<TEXTFILE>'       #  формат: id#####text\n


In [ ]:
ppl = []

with open(filename, 'r', encoding='utf-8') as f:
    itms = f.read().split('\n')
    for itm in itms:
        pers = itm.split('#####')[0]
        ppl.append(pers)
        

In [ ]:
ppl = list(set(ppl))


In [ ]:
users_get_url = 'https://api.vk.com/method/users.get'
all_users_data = []

for i in tqdm(range(len(ppl))):
    if ppl[i]:
        user_info = requests.get(
            users_get_url,
            params={
                'user_ids': int(ppl[i]),
                'fields': 'city,sex,bdate',
                'access_token': TOKEN,
                'v': VERSION
            }
        ).json()
        try:
            all_users_data.extend(user_info['response'])
        except:
            KeyError   # при случаях, когда автор комментария - сообщество
    

In [ ]:
cities, sex, bdate, ids = [], [], [], []


In [ ]:
for prson in all_users_data:
    try:
        if prson['id'] not in ids:
            ids.append(prson['id'])
            try:
                cities.append(prson['city']['title'])
            except:
                KeyError
                cities.append('-')
            try:
                sex.append(prson['sex'])
            except:
                KeyError
                sex.append('-')
            try:
                bdate.append(prson['bdate'])
            except:
                KeyError
                bdate.append('-')
    except:
        KeyError
        pass

Соединение мета-информации с текстом в один csv-файл:

In [ ]:
with open(filename, 'r', encoding='utf-8') as idtext_file:
    idtext_list = idtext_file.read().split('\n')
    del idtext_list[-1]
    sorted_ids = []
    texts = []
    for _ in idtext_list:
        user_id = _.split('#####')[0]
        sorted_ids.append(user_id)
        txt = _.split('#####')
        del txt[0]
        texts.append(''.join(txt))


In [ ]:
with open('project.csv', 'a', encoding='utf-8') as csv_file:
    file_writer = csv.writer(csv_file, delimiter = "\t")
    file_writer.writerow(["Id", "Text", "City", "Sex", "Bdate"])
    for j in tqdm(range(len(sorted_ids))):
        for i in range(len(ids)):
            if str(sorted_ids[j]) == str(ids[i]):
                if len(str(bdate[i])) > 6:
                    file_writer.writerow([sorted_ids[j], texts[j], cities[i], sex[i], bdate[i]])
                else:
                    file_writer.writerow([sorted_ids[j], texts[j], cities[i], sex[i], '-'])
                    

# Поиск численности населения городов

In [ ]:
df_meta = pd.read_csv('project.csv', sep='\t')

In [ ]:
df_c = list(df_meta['City'])

In [ ]:
import geocoder
import pandas as pd
from tqdm.auto import tqdm
city_ppl = {}
df_meta = pd.read_csv('projectt.csv', sep='\t')


In [ ]:
cities = list(set(df_c))

for city in cities:
    g = geocoder.geonames(city, key='<USERNAME>', featureClass='A')
    ppltn = g.population
    city_ppl[city] = ppltn


In [ ]:
city_int = {}

for city in list(city_ppl):
    if city_ppl[city] == None: 
        city_int[city] = 0
    else: 
        city_int[city] = city_ppl[city]
        

In [ ]:
df_popl = []
for city in df_c:
    if city == '-':
        df_popl.append(0)
    else:
        try:
            df_popl.append(city_int[city])
        except:
            print('error')
            

In [ ]:
df_meta['City population'] = df_popl


# Подсчеты

In [ ]:
rslt_df_more1m = df_meta[df_meta['City population'] >= 1000000]
rslt_df_less1m = df_meta[df_meta['City population'] < 1000000]
rslt_df_w = df_meta[df_meta['Sex'] == 2]
rslt_df_m = df_meta[df_meta['Sex'] >= 1]    


In [ ]:
textmore = ' '.join(list(rslt_df_more1m['Text']))
textless = ' '.join(list(rslt_df_less1m['Text']))
textw = ' '.join(list(rslt_df_w['Text']))
textm = ' '.join(list(rslt_df_m['Text']))
texts = ' '.join(list(df_meta['Text']))


In [ ]:
def count_tkns(text):
    all_words = []
    bld = re.findall('(блядь)[^а-яё]?', text.lower())
    blt = re.findall('(блять)[^а-яё]?', text.lower())
    bldi = re.findall('(бляд[ьиея][юймх]?[и]?)[^а-яё]?', text.lower())
    tkns = re.findall('([а-яёa-z\-0-9]+)[^а-яёa-z]?', text.lower())
    for tkn in tkns:
        if not re.findall('id[0-9]+', tkn):
            all_words.append(tkn)
    return len(bld), len(blt), len(all_words)


In [ ]:
all_bld, all_blt, all_tkns = count_tkns(texts)
more1m_bld, more1m_blt, more1m_tkns = count_tkns(textmore)
less1m_bld, less1m_blt, less1m_tkns = count_tkns(textless)
w_bld, w_blt, w_tkns = count_tkns(textw)
m_bld, m_blt, m_tkns = count_tkns(textm)


Количество токенов во всем тексте и в четырех подвыборках:

In [ ]:
print(all_tkns, more1m_tkns, less1m_tkns, w_tkns, m_tkns)


Процентное соотношение _блядь_ и _блять_ в городах с численностью населения более 1 миллиона и менее 1 миллиона:

In [ ]:
print(more1m_bld*100/(more1m_bld+more1m_blt), more1m_blt*100/(more1m_bld+more1m_blt))
print(less1m_bld*100/(less1m_bld+less1m_blt), less1m_blt*100/(less1m_bld+less1m_blt))


Процентное соотношение _блядь_ и _блять_ относительно пола комментатора:

In [ ]:
print(w_bld*100/(w_bld+w_blt), w_blt*100/(w_bld+w_blt))
print(m_blt*100/(m_blt+m_bld), m_bld*100/(m_blt+m_bld))


# Визуализации

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import random
random.seed = 23


In [ ]:
blt_w, bld_w = w_blt*100/(w_bld+w_blt), w_bld*100/(w_bld+w_blt)
blt_m, bld_m = m_blt*100/(m_blt+m_bld), m_bld*100/(m_blt+m_bld)


In [ ]:
X = [1, 3]
X2 = [2, 4]
Y = [blt_w, bld_w]
Y2 = [blt_m, bld_m]
TITLES = ['блять (ж)', 'блядь (ж)', 'блять (м)', 'блядь (м)']
plt.bar(X, Y, color='darkseagreen')
plt.bar(X2, Y2, color='darkcyan')
plt.xticks(ticks=X+X2, labels=TITLES)
plt.title('Распределение словоформ по полу\nговорящего в процентном соотношении')

plt.show()


In [ ]:
blt_20, bld_20, blt_midle, bld_midle, blt_30, bld_30 = 91, 9, 85, 15, 86, 14


In [ ]:
X = [1, 4]
X2 = [2, 5]
X3 = [3, 6]
Y = [blt_20, bld_20]
Y2 = [blt_midle, bld_midle]
Y3 = [blt_30, bld_30]
TITLES = ['до 20', 'до 20', 'от 20 до 30', 'от 20 до 30', 'до 30', 'до 30']
plt.bar(X, Y, color='darkseagreen')
plt.bar(X2, Y2, color='darkcyan')
plt.bar(X3, Y3, color='cadetblue')
plt.xticks(ticks=X+X2+X3, labels=TITLES)
plt.title('Распределение словоформ по возрасту\nговорящего в процентном соотношении')
plt.xlabel('блять                                         блядь')

plt.show()


In [ ]:
X = [2, 3, 4]
Y = [all_blt, 16181, all_bld]
TITLES = ['блять', 'блядь(общ.)', 'блядь-NOM']
plt.bar(X, Y, color='cadetblue')
plt.xticks(ticks=X, labels=TITLES)
plt.title('Общее количество вхождений различных словоформ')
plt.ylabel('Число вхождений')

plt.show()


In [ ]:
X = [1, 3]
X2 = [2, 4]
Y = [less1m_blt, less1m_bld]
Y2 = [more1m_blt, more1m_bld]
TITLES = ['блять (>1m)', 'блядь (>1m)', 'блять (<1m)', 'блядь (<1m)']
plt.bar(X, Y, color='cadetblue')
plt.bar(X2, Y2, color='darkseagreen')
plt.xticks(ticks=X+X2, labels=TITLES)
plt.title('Распределение словоформ по численности населения\nгорода говорящего в процентном соотношении')

plt.show()
